Ingénierie des caractéristiques et sélection des caractéristiques 

Dans cette section, nous commencerons par les connaissances et les idées acquises grâce à l’analyse exploratoire des données pour créer de bons ensembles de caractéristiques à l’aide de l’ingénierie des caractéristiques. 

Une chose à noter ici est que la plupart des fonctionnalités générées sont basées sur la connaissance du domaine et des expérimentations. Sans de bons ensembles de fonctionnalités, les algorithmes d’apprentissage automatique ne peuvent pas produire de résultats efficaces. Par conséquent, nous devrons trouver des moyens ingénieux de faire de l’ingénierie des caractéristiques pour que le modèle prenne de meilleures décisions. 

Outre la génération de fonctionnalités importantes/utiles, il est également essentiel de supprimer les fonctionnalités redondantes et bruyantes. Si nous introduisons des déchets/du bruit dans notre modèle d’apprentissage automatique, il renverra par conséquent des déchets/du bruit. Nous devons donc trouver de bonnes techniques de sélection de caractéristiques.

Modelisation

Une fois que nous en aurons terminé avec l’ingénierie des fonctionnalités, nous passerons à la modélisation de l’apprentissage automatique. Nous commencerons par des algorithmes simples mais puissants comme la régression logistique, la SVM linéaire, etc., puis nous passerons à des ensembles tels que les forêts aléatoires, les techniques de boosting, et enfin les techniques d’empilement. Nous choisirons les modèles qui donnent le meilleur résultat.

In [5]:
#importing Useful applicatication_trainStructures
import pandas as pd
import numpy as np
from scipy.stats import uniform

#importing plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns
from prettytable import PrettyTable

#importing Misc Libraries
import os
import gc
import pickle
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime

#for 100% jupyter notebook cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#sklearn
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split,StratifiedKFold,RandomizedSearchCV
from sklearn.metrics import roc_auc_score,precision_score,recall_score,roc_curve,confusion_matrix

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.calibration import CalibratedClassifierCV

#other modelling libraries
from bayes_opt import BayesianOptimization
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import XGBRegressor

import lightgbm as lgb
from lightgbm import LGBMClassifier
from lightgbm import LGBMRegressor

### process bureau et bureau_balance

bureau_balance.csv et bureau.csv
Ces tableaux contiennent les informations relatives aux crédits antérieurs du client qui n’étaient pas avec Home Credit Group, et ont été déclarés par le service de l’agence d’évaluation du crédit.


bureau_balance
Tout d’abord, la table bureau_balance contient trois champs, c’est-à-dire SK_ID_BUREAU, MONTHS_BALANCE et STATUS.
Comme le Status suit un comportement quelque peu ordinal, nous commençons par l’étiqueter en l’encodant.

Ensuite, certaines entités sont créées, telles que l’état pondéré, qui est obtenu en divisant l’état par le MONTHS_BALANCE.

Étant donné que les données contiennent la série chronologique, nous calculons également la moyenne mobile pondérée exponentielle des champs Statut et Statut pondéré.

Enfin, nous agrégeons les données sur SK_ID_BUREAU, de telle sorte que nous les agrégeons d’abord sur toutes les données, puis nous les agrégeons également sur les 2 dernières années. Ces 2 années dépeindraient le comportement plus récent des clients.

Les agrégations effectuées sont basées sur la connaissance du domaine, telles que la moyenne, le minimum, le maximum, la somme, le nombre, etc. Pour les fonctionnalités EDA, nous ne prenons que les dernières valeurs/les plus récentes, car elles contiennent en quelque sorte la tendance de toutes les valeurs précédentes.



bureau
Tout d’abord, nous fusionnons la table bureau avec la table agrégée bureau_balance de l’étape précédente, sur SK_ID_BUREAU.

Nous remplaçons certaines valeurs erronées par des valeurs NaN. Nous avons vu des prêts qui remontent à il y a 100 ans. Nous pensons qu’ils n’en diront pas grand-chose sur le comportement récent du client, alors nous les supprimons et ne conservons les prêts que sur une période de 50 ans.

Nous créons certaines caractéristiques par multiplications, divisions, soustractions de caractéristiques brutes, en fonction de la connaissance du domaine, telles que la durée du crédit, le ratio annuité/crédit, etc.

Les données catégorielles sont one hot encodées.

Pour les fusionner dans la table principale, c’est-à-dire application_train, nous agrégeons cette table sur SK_ID_CURR. Nous effectuons à nouveau les agrégations de deux manières. Nous agrégeons les crédits en fonction de la catégorie CREDIT_ACTIVE, où nous agrégeons séparément les deux catégories les plus populaires, c’est-à-dire Actif et Fermé. Plus tard, nous agrégeons également les catégories restantes et les fusionnons. Nous avons également agrégé l’ensemble des données. Les agrégations effectuées sont la somme, la moyenne, le min, le max, le dernier, etc.


##### BUREAU_BALANCE

In [21]:
bureau_balance = pd.read_csv('application_train/bureau_balance.csv')

In [22]:
dict_for_status = { 'C': 0, '0': 1, '1': 2, '2': 3, 'X': 4, '3': 5, '4': 6, '5': 7}
bureau_balance['STATUS'] = bureau_balance['STATUS'].map(dict_for_status)


In [23]:
#weighing the status with the months_balance
#converting months to positive
bureau_balance['MONTHS_BALANCE'] = np.abs(bureau_balance['MONTHS_BALANCE'])
bureau_balance['WEIGHTED_STATUS'] = bureau_balance.STATUS / (bureau_balance.MONTHS_BALANCE + 1)

# pas compris


In [25]:

#sorting the bureau_balance in ascending order of month and by the bureau SK_ID
#this is done so as to make the rolling exponential average easily for previous months till current month
bureau_balance = bureau_balance.sort_values(by=['SK_ID_BUREAU', 'MONTHS_BALANCE'], ascending=[0, 0])
        

In [26]:
#we will do exponential weighted average on the encoded status
#this is because if a person had a bad status 2 years ago, it should be given less weightage today
# we keep the latent variable alpha = 0.8 
#doing this for both weighted status and the status itself
bureau_balance['EXP_WEIGHTED_STATUS'] = bureau_balance.groupby('SK_ID_BUREAU')['WEIGHTED_STATUS'].transform(lambda x: x.ewm(alpha = 0.8).mean())
bureau_balance['EXP_ENCODED_STATUS'] = bureau_balance.groupby('SK_ID_BUREAU')['STATUS'].transform(lambda x: x.ewm(alpha = 0.8).mean())    


In [27]:
#we can see that these application_trainpoints are for 96 months i.e. 8 years.
#so we will extract the means, and exponential averages for each year separately
#first we convert month to year
bureau_balance['MONTHS_BALANCE'] = bureau_balance['MONTHS_BALANCE'] // 12

#On peut voir que ces application_trainpoints sont pour 96 mois, c’est-à-dire 8 ans.
#Nous allons donc extraire les moyennes et les moyennes exponentielles pour chaque année séparément
#Tout d’abord, nous convertissons le mois en année


In [28]:
#defining our aggregations
aggregations_basic = {
    'MONTHS_BALANCE' : ['mean','max'],
    'STATUS' : ['mean','max','first'],
    'WEIGHTED_STATUS' : ['mean','sum','first'],
    'EXP_ENCODED_STATUS' : ['last'],
    'EXP_WEIGHTED_STATUS' : ['last']}

In [29]:
#we will be finding aggregates for each year too
aggregations_for_year = {
    'STATUS' : ['mean','max','last','first'],
    'WEIGHTED_STATUS' : ['mean','max', 'first','last'],
    'EXP_WEIGHTED_STATUS' : ['last'],
    'EXP_ENCODED_STATUS' : ['last'] }

In [30]:
#aggregating over whole set first
aggregated_bureau_balance = bureau_balance.groupby(['SK_ID_BUREAU']).agg(aggregations_basic)
aggregated_bureau_balance.columns = ['_'.join(ele).upper() for ele in aggregated_bureau_balance.columns]


In [31]:
aggregated_bureau_balance

,MONTHS_BALANCE_MEAN,MONTHS_BALANCE_MAX,STATUS_MEAN,STATUS_MAX,STATUS_FIRST,WEIGHTED_STATUS_MEAN,WEIGHTED_STATUS_SUM,WEIGHTED_STATUS_FIRST,EXP_ENCODED_STATUS_LAST,EXP_WEIGHTED_STATUS_LAST
SK_ID_BUREAU,,,,,,,,,,
5001709,3.546392,8,0.453608,4,4,0.004936,0.478827,0.041237,3.094850e-60,3.547249e-62
5001710,2.963855,6,1.506024,4,4,0.022689,1.883185,0.048193,4.166370e-34,8.419081e-36
5001711,0.000000,0,1.750000,4,1,1.270833,5.083333,0.250000,3.403846e+00,3.297543e+00
5001712,0.368421,1,0.526316,1,1,0.037830,0.718771,0.052632,5.119999e-07,5.008223e-08
5001713,0.454545,1,4.000000,4,4,0.671057,14.763253,0.181818,4.000000e+00,3.570297e+00
...,...,...,...,...,...,...,...,...,...,...
6842884,1.500000,3,1.770833,4,4,0.058626,2.814045,0.083333,4.194304e-14,1.975070e-15
6842885,0.500000,1,4.000000,7,1,0.933134,22.395222,0.041667,7.000000e+00,6.248019e+00
6842886,0.909091,2,0.242424,1,1,0.008268,0.272840,0.030303,3.355435e-18,1.278811e-19


In [32]:
#aggregating some of the features separately for latest 2 years
aggregated_bureau_years = pd.Frame()
for year in range(2):
    year_group = bureau_balance[bureau_balance['MONTHS_BALANCE'] == year].groupby('SK_ID_BUREAU').agg(aggregations_for_year)
    year_group.columns = ['_'.join(ele).upper() + '_YEAR_' + str(year) for ele in year_group.columns]
    if year == 0:
        aggregated_bureau_years = year_group
    else:
        aggregated_bureau_years = aggregated_bureau_years.merge(year_group, on = 'SK_ID_BUREAU', how = 'outer')


In [34]:
#aggregating for rest of the years
aggregated_bureau_rest_years = bureau_balance[bureau_balance.MONTHS_BALANCE > year].groupby(['SK_ID_BUREAU']).agg(aggregations_for_year)
aggregated_bureau_rest_years.columns = ['_'.join(ele).upper() + '_YEAR_REST' for ele in aggregated_bureau_rest_years.columns]


In [35]:
#merging with rest of the years
aggregated_bureau_years = aggregated_bureau_years.merge(aggregated_bureau_rest_years, on = 'SK_ID_BUREAU', how = 'outer')
aggregated_bureau_balance = aggregated_bureau_balance.merge(aggregated_bureau_years, on = 'SK_ID_BUREAU', how = 'inner')


In [36]:
#filling the missing values obtained after aggregations with 0
aggregated_bureau_balance.fillna(0, inplace = True)

##### BUREAU

In [37]:
#### bureau
bureau = pd.read_csv('/bureau.csv')

In [38]:
#merging it with aggregated bureau_balance on 'SK_ID_BUREAU'
bureau_merged = bureau.merge(aggregated_bureau_balance, on = 'SK_ID_BUREAU', how = 'left')


In [39]:
#from the EDA we saw some erroneous values in DAYS Fields, we will remove those
#there are some loans which ended about very long ago, around 100 years ago.
#Thus we will only keep those loans which have ended in past 50 years.
bureau_merged['DAYS_CREDIT_ENDDATE'][bureau_merged['DAYS_CREDIT_ENDDATE'] > -50*365] = np.nan
bureau_merged['DAYS_ENDDATE_FACT'][bureau_merged['DAYS_ENDDATE_FACT'] > -50*365] = np.nan
#there is also a feature which tells about the number of days ago the Credit Report Came
bureau_merged['DAYS_CREDIT_UPDATE'][bureau_merged['DAYS_CREDIT_UPDATE'] > -50*365] = np.nan

#engineering some features based on domain knowledge
bureau_merged['CREDIT_DURATION'] = np.abs(bureau_merged['DAYS_CREDIT'] - bureau_merged['DAYS_CREDIT_ENDDATE'])
bureau_merged['FLAG_OVERDUE_RECENT'] = [0 if ele == 0 else 1 for ele in bureau_merged['CREDIT_DAY_OVERDUE']]
bureau_merged['MAX_AMT_OVERDUE_DURATION_RATIO'] = bureau_merged['AMT_CREDIT_MAX_OVERDUE'] / (bureau_merged['CREDIT_DURATION'] + 0.00001)
bureau_merged['CURRENT_AMT_OVERDUE_DURATION_RATIO'] = bureau_merged['AMT_CREDIT_SUM_OVERDUE'] / (bureau_merged['CREDIT_DURATION'] + 0.00001)
bureau_merged['AMT_OVERDUE_DURATION_LEFT_RATIO'] = bureau_merged['AMT_CREDIT_SUM_OVERDUE'] / (bureau_merged['DAYS_CREDIT_ENDDATE'] + 0.00001)
bureau_merged['CNT_PROLONGED_MAX_OVERDUE_MUL'] = bureau_merged['CNT_CREDIT_PROLONG'] * bureau_merged['AMT_CREDIT_MAX_OVERDUE']
bureau_merged['CNT_PROLONGED_DURATION_RATIO'] = bureau_merged['CNT_CREDIT_PROLONG'] / (bureau_merged['CREDIT_DURATION'] + 0.00001)
bureau_merged['CURRENT_DEBT_TO_CREDIT_RATIO'] = bureau_merged['AMT_CREDIT_SUM_DEBT'] / (bureau_merged['AMT_CREDIT_SUM'] + 0.00001)
bureau_merged['CURRENT_CREDIT_DEBT_DIFF'] = bureau_merged['AMT_CREDIT_SUM'] - bureau_merged['AMT_CREDIT_SUM_DEBT']
bureau_merged['AMT_ANNUITY_CREDIT_RATIO'] = bureau_merged['AMT_ANNUITY'] / (bureau_merged['AMT_CREDIT_SUM'] + 0.00001)
bureau_merged['CREDIT_ENDDATE_UPDATE_DIFF'] = np.abs(bureau_merged['DAYS_CREDIT_UPDATE'] - bureau_merged['DAYS_CREDIT_ENDDATE'])

In [40]:
#now we will be aggregating the bureau_merged df with respect to 'SK_ID_CURR' so as to merge it with application_train later    
#firstly we will aggregate the columns based on the category of CREDIT_ACTIVE
aggregations_CREDIT_ACTIVE = {
                'DAYS_CREDIT' : ['mean','min','max','last'],
                'CREDIT_DAY_OVERDUE' : ['mean','max'],
                'DAYS_CREDIT_ENDDATE' : ['mean','max'],
                'DAYS_ENDDATE_FACT' : ['mean','min'],
                'AMT_CREDIT_MAX_OVERDUE': ['max','sum'],
                'CNT_CREDIT_PROLONG': ['max','sum'],
                'AMT_CREDIT_SUM' : ['sum','max'],
                'AMT_CREDIT_SUM_DEBT': ['sum'],
                'AMT_CREDIT_SUM_LIMIT': ['max','sum'],
                'AMT_CREDIT_SUM_OVERDUE': ['max','sum'],
                'DAYS_CREDIT_UPDATE' : ['mean','min'],
                'AMT_ANNUITY' : ['mean','sum','max'],
                'CREDIT_DURATION' : ['max','mean'],
                'FLAG_OVERDUE_RECENT': ['sum'],
                'MAX_AMT_OVERDUE_DURATION_RATIO' : ['max','sum'],
                'CURRENT_AMT_OVERDUE_DURATION_RATIO' : ['max','sum'],
                'AMT_OVERDUE_DURATION_LEFT_RATIO' : ['max', 'mean'],
                'CNT_PROLONGED_MAX_OVERDUE_MUL' : ['mean','max'],
                'CNT_PROLONGED_DURATION_RATIO' : ['mean', 'max'],
                'CURRENT_DEBT_TO_CREDIT_RATIO' : ['mean', 'min'],
                'CURRENT_CREDIT_DEBT_DIFF' : ['mean','min'],
                'AMT_ANNUITY_CREDIT_RATIO' : ['mean','max','min'],
                'CREDIT_ENDDATE_UPDATE_DIFF' : ['max','min'],
                'STATUS_MEAN' : ['mean', 'max'],
                'WEIGHTED_STATUS_MEAN' : ['mean', 'max']
                 }

In [41]:
#we saw from EDA that the two most common type of CREDIT ACTIVE were 'Closed' and 'Active'.
#So we will aggregate them two separately and the remaining categories separately.
categories_to_aggregate_on = ['Closed','Active']
bureau_merged_aggregated_credit = pd.Frame()
for i, status in enumerate(categories_to_aggregate_on):
    group = bureau_merged[bureau_merged['CREDIT_ACTIVE'] == status].groupby('SK_ID_CURR').agg(aggregations_CREDIT_ACTIVE)
    group.columns = ['_'.join(ele).upper() + '_CREDITACTIVE_' + status.upper() for ele in group.columns]
    if i==0:
        bureau_merged_aggregated_credit = group
    else:
        bureau_merged_aggregated_credit = bureau_merged_aggregated_credit.merge(group, on = 'SK_ID_CURR', how = 'outer')


In [42]:
#aggregating for remaining categories
bureau_merged_aggregated_credit_rest = bureau_merged[(bureau_merged['CREDIT_ACTIVE'] != 'Active') & 
                                                     (bureau_merged['CREDIT_ACTIVE'] != 'Closed')].groupby('SK_ID_CURR').agg(aggregations_CREDIT_ACTIVE)
bureau_merged_aggregated_credit_rest.columns = ['_'.join(ele).upper() + 'CREDIT_ACTIVE_REST' for ele in bureau_merged_aggregated_credit_rest.columns]

In [43]:
 #merging with other categories
bureau_merged_aggregated_credit = bureau_merged_aggregated_credit.merge(bureau_merged_aggregated_credit_rest, on = 'SK_ID_CURR', how = 'outer')


In [44]:
#Encoding the categorical columns in one-hot form
currency_ohe = pd.get_dummies(bureau_merged['CREDIT_CURRENCY'], prefix = 'CURRENCY')
credit_active_ohe = pd.get_dummies(bureau_merged['CREDIT_ACTIVE'], prefix = 'CREDIT_ACTIVE')
credit_type_ohe = pd.get_dummies(bureau_merged['CREDIT_TYPE'], prefix = 'CREDIT_TYPE')


In [45]:
#merging the one-hot encoded columns
bureau_merged = pd.concat([bureau_merged.drop(['CREDIT_CURRENCY','CREDIT_ACTIVE','CREDIT_TYPE'], axis = 1), 
                           currency_ohe, credit_active_ohe, credit_type_ohe], axis = 1)


In [46]:
#aggregating the bureau_merged over all the columns
bureau_merged_aggregated = bureau_merged.drop('SK_ID_BUREAU', axis = 1).groupby('SK_ID_CURR').agg('mean')
bureau_merged_aggregated.columns = [ele + '_MEAN_OVERALL' for ele in bureau_merged_aggregated.columns]
#merging it with aggregates over categories
bureau_merged_aggregated = bureau_merged_aggregated.merge(bureau_merged_aggregated_credit, on = 'SK_ID_CURR', how = 'outer')


In [47]:
bureau_merged_aggregated

,DAYS_CREDIT_MEAN_OVERALL,CREDIT_DAY_OVERDUE_MEAN_OVERALL,DAYS_CREDIT_ENDDATE_MEAN_OVERALL,DAYS_ENDDATE_FACT_MEAN_OVERALL,AMT_CREDIT_MAX_OVERDUE_MEAN_OVERALL,CNT_CREDIT_PROLONG_MEAN_OVERALL,AMT_CREDIT_SUM_MEAN_OVERALL,AMT_CREDIT_SUM_DEBT_MEAN_OVERALL,AMT_CREDIT_SUM_LIMIT_MEAN_OVERALL,AMT_CREDIT_SUM_OVERDUE_MEAN_OVERALL,...,CURRENT_CREDIT_DEBT_DIFF_MINCREDIT_ACTIVE_REST,AMT_ANNUITY_CREDIT_RATIO_MEANCREDIT_ACTIVE_REST,AMT_ANNUITY_CREDIT_RATIO_MAXCREDIT_ACTIVE_REST,AMT_ANNUITY_CREDIT_RATIO_MINCREDIT_ACTIVE_REST,CREDIT_ENDDATE_UPDATE_DIFF_MAXCREDIT_ACTIVE_REST,CREDIT_ENDDATE_UPDATE_DIFF_MINCREDIT_ACTIVE_REST,STATUS_MEAN_MEANCREDIT_ACTIVE_REST,STATUS_MEAN_MAXCREDIT_ACTIVE_REST,WEIGHTED_STATUS_MEAN_MEANCREDIT_ACTIVE_REST,WEIGHTED_STATUS_MEAN_MAXCREDIT_ACTIVE_REST
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-735.000000,0.0,NaN,NaN,NaN,0.000000,2.076236e+05,85240.928571,0.00000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100002,-874.000000,0.0,NaN,NaN,1681.029,0.000000,1.081319e+05,49156.200000,7997.14125,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100003,-1400.750000,0.0,NaN,NaN,0.000,0.000000,2.543501e+05,0.000000,202500.00000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100004,-867.000000,0.0,NaN,NaN,0.000,0.000000,9.451890e+04,0.000000,0.00000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100005,-190.666667,0.0,NaN,NaN,0.000,0.000000,2.190420e+05,189469.500000,0.00000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456249,-1667.076923,0.0,NaN,NaN,6147.000,0.000000,2.841430e+05,16307.100000,0.00000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
456250,-862.000000,0.0,NaN,NaN,0.000,0.000000,1.028820e+06,744013.365000,19422.79500,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
456253,-867.500000,0.0,NaN,NaN,NaN,0.000000,9.900000e+05,448958.250000,0.00000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### PREVIOUS_APPLICATION

Ce tableau contient les données statiques relatives aux clients et à leurs crédits antérieurs auprès de Home Credit Group.

Tout d’abord, nous commençons par nettoyer les valeurs erronées. À partir de l’EDA, nous avons vu certains champs DAYS avec une valeur égale à 365243.0, ils semblent erronés, et nous allons donc les remplacer par des valeurs NaN.

Nous remplaçons les valeurs NaN des catégories par une catégorie 'XNA'.

Ensuite, nous procédons à l’ingénierie des fonctionnalités, où nous créons des fonctionnalités basées sur le domaine, telles que le ratio crédit-acompte, le montant non approuvé, le ratio crédit/marchandises, etc.

Nous essayons également de prédire le taux d’intérêt, en nous inspirant de l’un des articles des gagnants.

Pour pouvoir le fusionner avec la table principale, nous devons agréger les lignes de previous_application sur SK_ID_CURR. Nous effectuons des agrégations basées sur le domaine, sur tous les crédits précédents pour chaque client, tels que la moyenne, le maximum, le minimum, etc. Là encore, nous agrégeons de trois manières. Tout d’abord, nous effectuons l’agrégation globale, puis nous agrégeons pour les 2 premières applications et les 5 dernières applications. Le premier et le dernier sont décidés par le DAYS_FIRST_DUE des candidatures. En fin de compte, nous fusionnons toutes ces agrégations.

In [49]:
previpreviprevious_application = pd.read_csv('/previous_application.csv')

In [50]:
previous_application.shape

(1670214, 37)

In [51]:
#sorting the applications from oldest to most recent previous loans for each user
previous_application = previous_application.sort_values(by = ['SK_ID_CURR','DAYS_FIRST_DUE'])        

In [52]:
#in the EDA we found some erroneous values in DAYS columns, so we will replace them with NaN values
previous_application['DAYS_FIRST_DRAWING'][previous_application['DAYS_FIRST_DRAWING'] == 365243.0] = np.nan
previous_application['DAYS_FIRST_DUE'][previous_application['DAYS_FIRST_DUE'] == 365243.0] = np.nan
previous_application['DAYS_LAST_DUE_1ST_VERSION'][previous_application['DAYS_LAST_DUE_1ST_VERSION'] == 365243.0] = np.nan
previous_application['DAYS_LAST_DUE'][previous_application['DAYS_LAST_DUE'] == 365243.0] = np.nan
previous_application['DAYS_TERMINATION'][previous_application['DAYS_TERMINATION'] == 365243.0] = np.nan
#we also see abruptly large value for SELLERPLACE_AREA
previous_application['SELLERPLACE_AREA'][previous_application['SELLERPLACE_AREA'] == 4000000] = np.nan
#filling the NaN values for categories
categorical_columns = previous_application.dtypes[previous_application.dtypes == 'object'].index.tolist()
previous_application[categorical_columns] = previous_application[categorical_columns].fillna('XNA')
        

In [54]:
#label encoding the categorical variables
name_contract_dict = {'Approved': 0, 'Refused' : 3, 'Canceled' : 2, 'Unused offer' : 1}
previous_application['NAME_CONTRACT_STATUS'] = previous_application['NAME_CONTRACT_STATUS'].map(name_contract_dict)
yield_group_dict = {'XNA': 0, 'low_action': 1, 'low_normal': 2,'middle': 3, 'high': 4}
previous_application['NAME_YIELD_GROUP'] = previous_application['NAME_YIELD_GROUP'].map(yield_group_dict)
appl_per_contract_last_dict = {'Y':1, 'N':0}
previous_application['FLAG_LAST_APPL_PER_CONTRACT'] = previous_application['FLAG_LAST_APPL_PER_CONTRACT'].map(appl_per_contract_last_dict)
remaining_categorical_columns = previous_application.dtypes[previous_application.dtypes == 'object'].index.tolist()
for col in remaining_categorical_columns:
    encoding_dict = dict([(j,i) for i,j in enumerate(previous_application[col].unique(),1)])
    previous_application[col] = previous_application[col].map(encoding_dict)    


In [59]:
#engineering some features on domain knowledge
previous_application['MISSING_VALUES_TOTAL_PREV'] = previous_application.isna().sum(axis = 1)
previous_application['AMT_DECLINED'] = previous_application['AMT_APPLICATION'] - previous_application['AMT_CREDIT']
previous_application['AMT_CREDIT_GOODS_RATIO'] = previous_application['AMT_CREDIT'] / (previous_application['AMT_GOODS_PRICE'] + 0.00001)
previous_application['AMT__CREDIT_GOODS_DIFF'] = previous_application['AMT_CREDIT'] - previous_application['AMT_GOODS_PRICE']
previous_application['AMT_CREDIT_APPLICATION_RATIO'] = previous_application['AMT_APPLICATION'] / (previous_application['AMT_CREDIT'] + 0.00001)
previous_application['CREDIT_DOWNPAYMENT_RATIO'] = previous_application['AMT_DOWN_PAYMENT'] / (previous_application['AMT_CREDIT'] + 0.00001)
previous_application['GOOD_DOWNPAYMET_RATIO'] = previous_application['AMT_DOWN_PAYMENT'] / (previous_application['AMT_GOODS_PRICE'] + 0.00001)
previous_application['INTEREST_DOWNPAYMENT'] = previous_application['RATE_DOWN_PAYMENT'] * previous_application['AMT_DOWN_PAYMENT']
previous_application['INTEREST_CREDIT'] = previous_application['AMT_CREDIT'] * previous_application['RATE_INTEREST_PRIMARY']
previous_application['INTEREST_CREDIT_PRIVILEGED'] = previous_application['AMT_CREDIT'] * previous_application['RATE_INTEREST_PRIVILEGED']
previous_application['APPLICATION_AMT_TO_DECISION_RATIO'] = previous_application['AMT_APPLICATION'] / (previous_application['DAYS_DECISION'] + 0.00001) * -1
previous_application['AMT_APPLICATION_TO_SELLERPLACE_AREA'] = previous_application['AMT_APPLICATION'] / (previous_application['SELLERPLACE_AREA'] + 0.00001)
previous_application['ANNUITY'] = previous_application['AMT_CREDIT'] / (previous_application['CNT_PAYMENT'] + 0.00001)
previous_application['ANNUITY_GOODS'] = previous_application['AMT_GOODS_PRICE'] / (previous_application['CNT_PAYMENT'] + 0.00001)
previous_application['DAYS_FIRST_LAST_DUE_DIFF' ] = previous_application['DAYS_LAST_DUE'] - previous_application['DAYS_FIRST_DUE']
previous_application['AMT_CREDIT_HOUR_PROCESS_START'] = previous_application['AMT_CREDIT'] * previous_application['HOUR_APPR_PROCESS_START']
previous_application['AMT_CREDIT_NFLAG_LAST_APPL_DAY'] = previous_application['AMT_CREDIT'] * previous_application['NFLAG_LAST_APPL_IN_DAY']
previous_application['AMT_CREDIT_YIELD_GROUP'] = previous_application['AMT_CREDIT'] * previous_application['NAME_YIELD_GROUP']
#https://www.kaggle.com/c/home-credit-default-risk/discussion/64598
previous_application['AMT_INTEREST'] = previous_application['CNT_PAYMENT'] * previous_application['AMT_ANNUITY'] - previous_application['AMT_CREDIT'] 
previous_application['INTEREST_SHARE'] = previous_application['AMT_INTEREST'] / (previous_application['AMT_CREDIT'] + 0.00001)
previous_application['INTEREST_RATE'] = 2 * 12 * previous_application['AMT_INTEREST'] / (previous_application['AMT_CREDIT'] * (previous_application['CNT_PAYMENT'] + 1))


In [63]:
aggregations_for_previous_application = {
            'MISSING_VALUES_TOTAL_PREV' : ['sum'],
            'NAME_CONTRACT_TYPE' : ['mean','last'],
            'AMT_ANNUITY' : ['mean','sum','max'],
            'AMT_APPLICATION' : ['mean','max','sum'],
            'AMT_CREDIT' : ['mean','max','sum'],
            'AMT_DOWN_PAYMENT' : ['mean','max','sum'],
            'AMT_GOODS_PRICE' : ['mean','max','sum'],
            'WEEKDAY_APPR_PROCESS_START' : ['mean','max','min'],
            'HOUR_APPR_PROCESS_START' : ['mean','max','min'],
            'FLAG_LAST_APPL_PER_CONTRACT' : ['mean','sum'],
            'NFLAG_LAST_APPL_IN_DAY' : ['mean','sum'],
            'RATE_DOWN_PAYMENT' : ['mean','max'],
            'RATE_INTEREST_PRIMARY' : ['mean','max'],
            'RATE_INTEREST_PRIVILEGED' : ['mean','max'],
            'NAME_CASH_LOAN_PURPOSE' : ['mean','last'],
            'NAME_CONTRACT_STATUS' : ['mean','max','last'],
            'DAYS_DECISION' : ['mean','max','min'],
            'NAME_PAYMENT_TYPE' : ['mean', 'last'],
            'CODE_REJECT_REASON' : ['mean','last'],
            'NAME_TYPE_SUITE' : ['mean','last'],
            'NAME_CLIENT_TYPE' : ['mean','last'],
            'NAME_GOODS_CATEGORY' : ['mean','last'],
            'NAME_PORTFOLIO' : ['mean','last'],
            'NAME_PRODUCT_TYPE' : ['mean','last'],
            'CHANNEL_TYPE' : ['mean','last'],
            'SELLERPLACE_AREA' : ['mean','max','min'],
            'NAME_SELLER_INDUSTRY' : ['mean','last'],
            'CNT_PAYMENT' : ['sum','mean','max'],
            'NAME_YIELD_GROUP' : ['mean','last'],
            'PRODUCT_COMBINATION' : ['mean', 'last'],
            'DAYS_FIRST_DRAWING' : ['mean','max'],
            'DAYS_FIRST_DUE' : ['mean','max'],
            'DAYS_LAST_DUE_1ST_VERSION' : ['mean'],
            'DAYS_LAST_DUE' : ['mean'],
            'DAYS_TERMINATION' : ['mean','max'],
            'NFLAG_INSURED_ON_APPROVAL' : ['sum'],
            'AMT_DECLINED' : ['mean','max','sum'],
            'AMT_CREDIT_GOODS_RATIO' : ['mean', 'max', 'min'],
            #'AMT_CREDIT_GOODS_DIFF' : ['sum','mean','max', 'min'],
            'AMT_CREDIT_APPLICATION_RATIO' : ['mean','min'],
            'CREDIT_DOWNPAYMENT_RATIO' : ['mean','max'],
            'GOOD_DOWNPAYMET_RATIO' : ['mean','max'],
            'INTEREST_DOWNPAYMENT' : ['mean','sum','max'],
            'INTEREST_CREDIT' : ['mean','sum','max'],
            'INTEREST_CREDIT_PRIVILEGED' : ['mean','sum','max'],
            'APPLICATION_AMT_TO_DECISION_RATIO' : ['mean','min'],
            'AMT_APPLICATION_TO_SELLERPLACE_AREA' : ['mean','max'],
            'ANNUITY' : ['mean','sum','max'],
            'ANNUITY_GOODS' : ['mean','sum','max'],
            'DAYS_FIRST_LAST_DUE_DIFF' : ['mean','max'],
            'AMT_CREDIT_HOUR_PROCESS_START' : ['mean','sum'],
            'AMT_CREDIT_NFLAG_LAST_APPL_DAY' : ['mean','max'],
            'AMT_CREDIT_YIELD_GROUP' : ['mean','sum','min'],
            'AMT_INTEREST' : ['mean','sum','max','min'],
            'INTEREST_SHARE' : ['mean','max','min'],
            'INTEREST_RATE' : ['mean','max','min']
        }

In [64]:
#grouping the previous applications over SK_ID_CURR while only taking the latest 5 applications
group_last_3 = previous_application.groupby('SK_ID_CURR').tail(5).groupby('SK_ID_CURR').agg(aggregations_for_previous_application)
group_last_3.columns = ['_'.join(ele).upper() + '_LAST_5' for ele in group_last_3.columns]
        

In [65]:
#grouping the previous applications over SK_ID_CURR while only taking the first 2 applications
group_first_3 = previous_application.groupby('SK_ID_CURR').head(2).groupby('SK_ID_CURR').agg(aggregations_for_previous_application)
group_first_3.columns = ['_'.join(ele).upper() + '_FIRST_2' for ele in group_first_3.columns]
        

In [67]:
#grouping the previous applications over SK_ID_CURR while taking all the applications into consideration
group_all = previous_application.groupby('SK_ID_CURR').agg(aggregations_for_previous_application)
group_all.columns = ['_'.join(ele).upper() + '_ALL' for ele in group_all.columns]


In [68]:

#merging all the applications
previous_application_aggregated = group_last_3.merge(group_first_3, on = 'SK_ID_CURR', how = 'outer')
previous_application_aggregated = previous_application_aggregated.merge(group_all, on = 'SK_ID_CURR', how = 'outer')


### application train et test

Ces tableaux sont constitués de données statiques relatives aux Emprunteurs. Chaque ligne représente une demande de prêt.

Tout d’abord, nous commençons par nettoyer les données en supprimant les points de données erronés. Nous supprimons également les lignes dans les données d’apprentissage avec des catégories de sorte que ces catégories n’apparaissent pas dans les données de test. Nous convertissons également les caractéristiques d’évaluation de la région en caractéristiques catégorielles, car nous avons vu dans l’EDA qu’elles ne suivent pas un comportement ordinal en ce qui concerne les caractéristiques par défaut.

En nous inspirant de l’article du gagnant pour le problème, nous prédisons également les valeurs manquantes des caractéristiques de EXT_SOURCE en construisant un modèle de régression sur le reste des caractéristiques numériques.

Ensuite, nous effectuons l’ingénierie des caractéristiques sur les caractéristiques numériques et générons des caractéristiques basées sur la connaissance du domaine, telles que le ratio REVENU / RENTE, EXT_SOURCE moyennes, etc.

Nous essayons également de prédire les taux d’intérêt en utilisant les données des fonctionnalités des applications précédentes, et en prédisant en utilisant les données de application_train fonctionnalités. Nous créons également une entité basée sur les valeurs Target à partir de application_train où nous calculons la moyenne des cibles de 500 voisins les plus proches de chaque ligne.

Ensuite, nous créons des caractéristiques basées sur les interactions catégorielles en regroupant les données sur plusieurs combinaisons catégorielles et en imputant les agrégats de chaque groupe en tant qu’entités.

Nous encodons les caractéristiques catégorielles par codage de réponse, car nous ne voulions pas augmenter la dimensionnalité de plusieurs fois en utilisant OHE.

In [71]:
application_train = pd.read_csv('/application_train.csv')
application_test = pd.read_csv('/application_test.csv')

In [72]:
flag_cols_to_drop = ['FLAG_DOCUMENT_2','FLAG_DOCUMENT_4','FLAG_DOCUMENT_10','FLAG_DOCUMENT_12',
                            'FLAG_DOCUMENT_20']

In [73]:
application_train = application_train.drop(flag_cols_to_drop, axis = 1)
application_test = application_test.drop(flag_cols_to_drop, axis = 1)
#converting age from days to years
application_train['DAYS_BIRTH'] = application_train['DAYS_BIRTH'] * -1 / 365
application_test['DAYS_BIRTH'] = application_test['DAYS_BIRTH'] * -1 / 365
        

In [74]:
#From the EDA we saw some erroneous values in DAYS_EMPLOYED field
application_train['DAYS_EMPLOYED'][application_train['DAYS_EMPLOYED'] == 365243] = np.nan
application_test['DAYS_EMPLOYED'][application_test['DAYS_EMPLOYED'] == 365243] = np.nan
        

In [75]:
#OBS Columns have an erroneous value, we'll remove those values
application_train['OBS_30_CNT_SOCIAL_CIRCLE'][application_train['OBS_30_CNT_SOCIAL_CIRCLE'] > 30] == np.nan
application_train['OBS_60_CNT_SOCIAL_CIRCLE'][application_train['OBS_60_CNT_SOCIAL_CIRCLE'] > 30] == np.nan
application_test['OBS_30_CNT_SOCIAL_CIRCLE'][application_test['OBS_30_CNT_SOCIAL_CIRCLE'] > 30] == np.nan
application_test['OBS_60_CNT_SOCIAL_CIRCLE'][application_test['OBS_60_CNT_SOCIAL_CIRCLE'] > 30] == np.nan


37465    False
38869    False
39671    False
Name: OBS_60_CNT_SOCIAL_CIRCLE, dtype: bool

In [76]:
#there were also 4 rows with 'XNA' as Gender, removing these rows
application_train = application_train[application_train['CODE_GENDER'] != 'XNA']
#filling the categorical columns with 'XNA' value
categorical_columns = application_train.dtypes[application_train.dtypes == 'object'].index.tolist()
application_train[categorical_columns] = application_train[categorical_columns].fillna('XNA')
application_test[categorical_columns] = application_test[categorical_columns].fillna('XNA')
       

In [77]:
#converting columns of REGION_RATING_CLIENT to object type, as we saw some complex impact on TARGET variable during EDA
application_train['REGION_RATING_CLIENT'] = application_train['REGION_RATING_CLIENT'].astype('object')
application_train['REGION_RATING_CLIENT_W_CITY'] = application_train['REGION_RATING_CLIENT_W_CITY'].astype('object')
application_test['REGION_RATING_CLIENT'] = application_test['REGION_RATING_CLIENT'].astype('object')
application_test['REGION_RATING_CLIENT_W_CITY'] = application_test['REGION_RATING_CLIENT_W_CITY'].astype('object')
#counting the total NaN values for each application
application_train['MISSING_VALS_TOTAL_APP'] = application_train.isna().sum(axis = 1)
application_test['MISSING_VALS_TOTAL_APP'] = application_test.isna().sum(axis = 1)


In [81]:
application_train['CREDIT_INCOME_RATIO'] = application_train['AMT_CREDIT'] / (application_train['AMT_INCOME_TOTAL'] + 0.00001)
application_train['CREDIT_ANNUITY_RATIO'] = application_train['AMT_CREDIT'] / (application_train['AMT_ANNUITY'] + 0.00001)
application_train['ANNUITY_INCOME_RATIO'] = application_train['AMT_ANNUITY'] / (application_train['AMT_INCOME_TOTAL'] + 0.00001)
application_train['INCOME_ANNUITY_DIFF'] = application_train['AMT_INCOME_TOTAL'] - application_train['AMT_ANNUITY']
application_train['CREDIT_GOODS_RATIO'] = application_train['AMT_CREDIT'] / (application_train['AMT_GOODS_PRICE'] + 0.00001)
application_train['CREDIT_GOODS_DIFF'] = application_train['AMT_CREDIT'] - application_train['AMT_GOODS_PRICE'] + 0.00001
application_train['GOODS_INCOME_RATIO'] = application_train['AMT_GOODS_PRICE'] / (application_train['AMT_INCOME_TOTAL'] + 0.00001)
application_train['INCOME_EXT_RATIO'] = application_train['AMT_INCOME_TOTAL'] / (application_train['EXT_SOURCE_3'] + 0.00001)
application_train['CREDIT_EXT_RATIO'] = application_train['AMT_CREDIT'] / (application_train['EXT_SOURCE_3'] + 0.00001)
#age ratios and diffs
application_train['AGE_EMPLOYED_DIFF'] = application_train['DAYS_BIRTH'] - application_train['DAYS_EMPLOYED']
application_train['EMPLOYED_TO_AGE_RATIO'] = application_train['DAYS_EMPLOYED'] / (application_train['DAYS_BIRTH'] + 0.00001)
#car ratios
application_train['CAR_EMPLOYED_DIFF'] = application_train['OWN_CAR_AGE'] - application_train['DAYS_EMPLOYED']
application_train['CAR_EMPLOYED_RATIO'] = application_train['OWN_CAR_AGE'] / (application_train['DAYS_EMPLOYED']+0.00001)
application_train['CAR_AGE_DIFF'] = application_train['DAYS_BIRTH'] - application_train['OWN_CAR_AGE']
application_train['CAR_AGE_RATIO'] = application_train['OWN_CAR_AGE'] / (application_train['DAYS_BIRTH'] + 0.00001)
#flag contacts sum
application_train['FLAG_CONTACTS_SUM'] = application_train['FLAG_MOBIL'] + application_train['FLAG_EMP_PHONE'] + application_train['FLAG_WORK_PHONE'] + application_train[
                            'FLAG_CONT_MOBILE'] + application_train['FLAG_PHONE'] + application_train['FLAG_EMAIL']



application_train['HOUR_PROCESS_CREDIT_MUL'] = application_train['AMT_CREDIT'] * application_train['HOUR_APPR_PROCESS_START']
#family members
application_train['CNT_NON_CHILDREN'] = application_train['CNT_FAM_MEMBERS'] - application_train['CNT_CHILDREN']
application_train['CHILDREN_INCOME_RATIO'] = application_train['CNT_CHILDREN'] / (application_train['AMT_INCOME_TOTAL'] + 0.00001)
application_train['PER_CAPITA_INCOME'] = application_train['AMT_INCOME_TOTAL'] / (application_train['CNT_FAM_MEMBERS'] + 1)
#region ratings
application_train['REGIONS_RATING_INCOME_MUL'] = (application_train['REGION_RATING_CLIENT'] + application_train['REGION_RATING_CLIENT_W_CITY']) * application_train['AMT_INCOME_TOTAL'] / 2
application_train['REGION_RATING_MAX'] = [max(ele1, ele2) for ele1, ele2 in zip(application_train['REGION_RATING_CLIENT'], application_train['REGION_RATING_CLIENT_W_CITY'])]
application_train['REGION_RATING_MAX'] = [min(ele1, ele2) for ele1, ele2 in zip(application_train['REGION_RATING_CLIENT'], application_train['REGION_RATING_CLIENT_W_CITY'])]
application_train['REGION_RATING_MEAN'] = (application_train['REGION_RATING_CLIENT'] + application_train['REGION_RATING_CLIENT_W_CITY']) / 2
application_train['REGION_RATING_MUL'] = application_train['REGION_RATING_CLIENT'] * application_train['REGION_RATING_CLIENT_W_CITY']
#flag regions
application_train['FLAG_REGIONS'] = application_train['REG_REGION_NOT_LIVE_REGION'] + application_train['REG_REGION_NOT_WORK_REGION'] + application_train['LIVE_REGION_NOT_WORK_REGION']+application_train[
                        'REG_CITY_NOT_LIVE_CITY'] + application_train['REG_CITY_NOT_WORK_CITY'] + application_train['LIVE_CITY_NOT_WORK_CITY']   
#ext_sources
application_train['EXT_SOURCE_MEAN'] = (application_train['EXT_SOURCE_1'] + application_train['EXT_SOURCE_2'] + application_train['EXT_SOURCE_3'] ) / 3
application_train['EXT_SOURCE_MUL'] = application_train['EXT_SOURCE_1'] * application_train['EXT_SOURCE_2'] * application_train['EXT_SOURCE_3'] 
application_train['EXT_SOURCE_MAX'] = [max(ele1,ele2,ele3) for ele1, ele2, ele3 in zip(application_train['EXT_SOURCE_1'], application_train['EXT_SOURCE_2'], application_train['EXT_SOURCE_3'])]
application_train['EXT_SOURCE_MIN'] = [min(ele1,ele2,ele3) for ele1, ele2, ele3 in zip(application_train['EXT_SOURCE_1'], application_train['EXT_SOURCE_2'], application_train['EXT_SOURCE_3'])]
application_train['EXT_SOURCE_VAR'] = [np.var([ele1,ele2,ele3]) for ele1, ele2, ele3 in zip(application_train['EXT_SOURCE_1'], application_train['EXT_SOURCE_2'], application_train['EXT_SOURCE_3'])]
application_train['WEIGHTED_EXT_SOURCE'] =  application_train.EXT_SOURCE_1 * 2 + application_train.EXT_SOURCE_2 * 3 + application_train.EXT_SOURCE_3 * 4
#apartment scores
application_train['APARTMENTS_SUM_AVG'] = application_train['APARTMENTS_AVG'] + application_train['BASEMENTAREA_AVG'] + application_train['YEARS_BEGINEXPLUATATION_AVG'] + application_train[
                            'YEARS_BUILD_AVG'] + application_train['COMMONAREA_AVG'] + application_train['ELEVATORS_AVG'] + application_train['ENTRANCES_AVG'] + application_train[
                            'FLOORSMAX_AVG'] + application_train['FLOORSMIN_AVG'] + application_train['LANDAREA_AVG'] + application_train['LIVINGAPARTMENTS_AVG'] + application_train[
                            'LIVINGAREA_AVG'] + application_train['NONLIVINGAPARTMENTS_AVG'] + application_train['NONLIVINGAREA_AVG']
application_train['APARTMENTS_SUM_MODE'] = application_train['APARTMENTS_MODE'] + application_train['BASEMENTAREA_MODE'] + application_train['YEARS_BEGINEXPLUATATION_MODE'] + application_train[
                            'YEARS_BUILD_MODE'] + application_train['COMMONAREA_MODE'] + application_train['ELEVATORS_MODE'] + application_train['ENTRANCES_MODE'] + application_train[
                            'FLOORSMAX_MODE'] + application_train['FLOORSMIN_MODE'] + application_train['LANDAREA_MODE'] + application_train['LIVINGAPARTMENTS_MODE'] + application_train[
                            'LIVINGAREA_MODE'] + application_train['NONLIVINGAPARTMENTS_MODE'] + application_train['NONLIVINGAREA_MODE'] + application_train['TOTALAREA_MODE']
application_train['APARTMENTS_SUM_MEDI'] = application_train['APARTMENTS_MEDI'] + application_train['BASEMENTAREA_MEDI'] + application_train['YEARS_BEGINEXPLUATATION_MEDI'] + application_train[
                            'YEARS_BUILD_MEDI'] + application_train['COMMONAREA_MEDI'] + application_train['ELEVATORS_MEDI'] + application_train['ENTRANCES_MEDI'] + application_train[
                            'FLOORSMAX_MEDI'] + application_train['FLOORSMIN_MEDI'] + application_train['LANDAREA_MEDI'] + application_train['LIVINGAPARTMENTS_MEDI'] + application_train[
                            'LIVINGAREA_MEDI'] + application_train['NONLIVINGAPARTMENTS_MEDI'] + application_train['NONLIVINGAREA_MEDI']
application_train['INCOME_APARTMENT_AVG_MUL'] = application_train['APARTMENTS_SUM_AVG'] * application_train['AMT_INCOME_TOTAL']
application_train['INCOME_APARTMENT_MODE_MUL'] = application_train['APARTMENTS_SUM_MODE'] * application_train['AMT_INCOME_TOTAL']
application_train['INCOME_APARTMENT_MEDI_MUL'] = application_train['APARTMENTS_SUM_MEDI'] * application_train['AMT_INCOME_TOTAL']
#OBS And DEF
application_train['OBS_30_60_SUM'] = application_train['OBS_30_CNT_SOCIAL_CIRCLE'] + application_train['OBS_60_CNT_SOCIAL_CIRCLE']
application_train['DEF_30_60_SUM'] = application_train['DEF_30_CNT_SOCIAL_CIRCLE'] + application_train['DEF_60_CNT_SOCIAL_CIRCLE']
application_train['OBS_DEF_30_MUL'] = application_train['OBS_30_CNT_SOCIAL_CIRCLE'] *  application_train['DEF_30_CNT_SOCIAL_CIRCLE']
application_train['OBS_DEF_60_MUL'] = application_train['OBS_60_CNT_SOCIAL_CIRCLE'] *  application_train['DEF_60_CNT_SOCIAL_CIRCLE']
application_train['SUM_OBS_DEF_ALL'] = application_train['OBS_30_CNT_SOCIAL_CIRCLE'] + application_train['DEF_30_CNT_SOCIAL_CIRCLE'] + application_train[
                            'OBS_60_CNT_SOCIAL_CIRCLE'] + application_train['DEF_60_CNT_SOCIAL_CIRCLE']
application_train['OBS_30_CREDIT_RATIO'] = application_train['AMT_CREDIT'] / (application_train['OBS_30_CNT_SOCIAL_CIRCLE'] + 0.00001)
application_train['OBS_60_CREDIT_RATIO'] = application_train['AMT_CREDIT'] / (application_train['OBS_60_CNT_SOCIAL_CIRCLE'] + 0.00001)
application_train['DEF_30_CREDIT_RATIO'] = application_train['AMT_CREDIT'] / (application_train['DEF_30_CNT_SOCIAL_CIRCLE'] + 0.00001)
application_train['DEF_60_CREDIT_RATIO'] = application_train['AMT_CREDIT'] / (application_train['DEF_60_CNT_SOCIAL_CIRCLE'] + 0.00001)
#Flag Documents combined
application_train['SUM_FLAGS_DOCUMENTS'] = application_train['FLAG_DOCUMENT_3'] + application_train['FLAG_DOCUMENT_5'] + application_train['FLAG_DOCUMENT_6']  + application_train[
                            'FLAG_DOCUMENT_7'] + application_train['FLAG_DOCUMENT_8'] + application_train['FLAG_DOCUMENT_9'] + application_train[
                            'FLAG_DOCUMENT_11'] + application_train['FLAG_DOCUMENT_13'] + application_train['FLAG_DOCUMENT_14'] + application_train[
                            'FLAG_DOCUMENT_15'] + application_train['FLAG_DOCUMENT_16'] + application_train['FLAG_DOCUMENT_17'] + application_train[
                            'FLAG_DOCUMENT_18'] + application_train['FLAG_DOCUMENT_19'] + application_train['FLAG_DOCUMENT_21']
#details change
application_train['DAYS_DETAILS_CHANGE_MUL'] = application_train['DAYS_LAST_PHONE_CHANGE'] * application_train['DAYS_REGISTRATION'] * application_train['DAYS_ID_PUBLISH']
application_train['DAYS_DETAILS_CHANGE_SUM'] = application_train['DAYS_LAST_PHONE_CHANGE'] + application_train['DAYS_REGISTRATION'] + application_train['DAYS_ID_PUBLISH']
#enquires
application_train['AMT_ENQ_SUM'] = application_train['AMT_REQ_CREDIT_BUREAU_HOUR'] + application_train['AMT_REQ_CREDIT_BUREAU_DAY'] + application_train['AMT_REQ_CREDIT_BUREAU_WEEK'] + application_train[
                    'AMT_REQ_CREDIT_BUREAU_MON'] + application_train['AMT_REQ_CREDIT_BUREAU_QRT'] + application_train['AMT_REQ_CREDIT_BUREAU_YEAR']
application_train['ENQ_CREDIT_RATIO'] = application_train['AMT_ENQ_SUM'] / (application_train['AMT_CREDIT'] + 0.00001)





In [82]:
application_train

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,SUM_OBS_DEF_ALL,OBS_30_CREDIT_RATIO,OBS_60_CREDIT_RATIO,DEF_30_CREDIT_RATIO,DEF_60_CREDIT_RATIO,SUM_FLAGS_DOCUMENTS,DAYS_DETAILS_CHANGE_MUL,DAYS_DETAILS_CHANGE_SUM,AMT_ENQ_SUM,ENQ_CREDIT_RATIO
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,8.0,2.032977e+05,2.032977e+05,2.032977e+05,2.032977e+05,1,-8.770084e+09,-6902.0,1.0,0.000002
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,2.0,1.293490e+06,1.293490e+06,1.293502e+11,1.293502e+11,1,-2.857643e+08,-2305.0,0.0,0.000000
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0.0,1.350000e+10,1.350000e+10,1.350000e+10,1.350000e+10,0,-8.787379e+09,-7606.0,0.0,0.000000
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,4.0,1.563405e+05,1.563405e+05,3.126825e+10,3.126825e+10,1,-1.478518e+10,-12887.0,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0.0,5.130000e+10,5.130000e+10,5.130000e+10,5.130000e+10,1,-1.648763e+10,-8875.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0.0,2.547000e+10,2.547000e+10,2.547000e+10,2.547000e+10,1,-4.575423e+09,-10711.0,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0.0,2.695500e+10,2.695500e+10,2.695500e+10,2.695500e+10,1,0.000000e+00,-8478.0,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,12.0,1.129438e+05,1.129438e+05,6.776640e+10,6.776640e+10,1,-6.623380e+10,-13796.0,3.0,0.000004
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0.0,3.701070e+10,3.701070e+10,3.701070e+10,3.701070e+10,1,-7.680415e+08,-3815.0,0.0,0.000000


In [ ]:
# merge tables
